In [84]:
import tweepy #requires pip install
import numpy as np
import pandas as pd
import config

In [85]:
consumer_key = config.consumer_key
consumer_secret = config.consumer_secret
access_token = config.access_token
access_token_secret = config.access_token_secret

In [42]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [43]:
stations = pd.read_csv ('/Users/christopherkindl/working/data-engineering-group-assignment/\
01_web_scraping/Transport Data.csv')

In [64]:
import re

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [61]:
text = 'This is a smiley face U+2764'
print(text)
deEmojify(text)

This is a smiley face U+2764


'This is a smiley face U+2764'

In [87]:
# test query
number_of_tweets = 10
tweets = []
station = []
date = []

number_of_stations = 30

for index in range(len(stations[:number_of_stations])):
    for i in tweepy.Cursor(api.search, q = 'london', lang = 'en', tweet_mode='extended', geocode= \
                           str(stations['Latitude'][index])+','+str(stations['Longitude'][index])+',1km').\
        items(number_of_tweets):
        tweets.append(i.full_text)
        station.append(stations['Station'][index])
        date.append(i.created_at)

In [88]:
df = pd.DataFrame({'tweets': tweets, 'date':date, 'station': station})

In [89]:
df

,tweets,date,station
0,@paintpeteuk @BBCRadio3 Have seen it performed...,2021-05-01 20:06:22,Abbey Road
1,"Just posted a photo @ Abbey Road, London, UK h...",2021-05-01 16:01:07,Abbey Road
2,"Abbey Road #throwback \n#2019\nLondon, UK @ Ab...",2021-04-30 03:23:21,Abbey Road
3,@martjdayo @robnitm It's back in London at Can...,2021-04-28 16:28:29,Abbey Road
4,@noordinarypark @WUParks @LeeValleyPark @AMOrb...,2021-04-27 07:30:07,Abbey Road
...,...,...,...
114,@Ana_Mitras @Uber If they had the knowledge th...,2021-05-01 15:31:09,Barkingside
115,"Labour brought in Devolution to Scotland, Wale...",2021-05-01 14:51:52,Barkingside
116,My new list of no go areas hackney. Islington ...,2021-05-01 08:09:17,Barkingside
117,@TfLTPH You have to ask why taxis have to over...,2021-05-01 07:01:08,Barkingside


In [72]:
df.to_csv("test-wed.tsv", sep="\t")

In [56]:
df.to_csv('/Users/christopherkindl/Desktop/wed_test_3.csv')

In [79]:
df.to_parquet('/Users/christopherkindl/Desktop/df_2.parquet')

In [77]:
df.to_parquet('/Users/christopherkindl/Desktop/df_2.parquet.gzip', compression='gzip')

In [59]:
# pyspark modules
from pyspark.sql import SparkSession
from textblob import TextBlob
from pyspark.sql.functions import udf

In [60]:
!pip install vaderSentiment

You should consider upgrading via the '/Users/christopherkindl/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [ ]:
def apply_blob(sentence):
    temp = TextBlob(sentence).sentiment[0]
    if temp == 0.0:
        return 0.0 # Neutral
    elif temp >= 0.0:
        return 1.0 # Positive
    else:
        return 2.0 # Negative

In [61]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
for sentence in sentences:
    vs = analyzer.polarity_scores(sentence)

In [63]:
sentence = 'Make sure that you dont sound to cooperate!'
vs = analyzer.polarity_scores(sentence)

In [71]:
vs

{'neg': 0.0, 'neu': 0.73, 'pos': 0.27, 'compound': 0.3802}

In [90]:
def apply_vader(sentence):
    vs = analyzer.polarity_scores(sentence)    
    return float(round(vs.get('compound'), 2))

# assign sentiment function as an user defined function
sentiment = udf(apply_vader)

In [91]:
spark = SparkSession.builder.appName('SentimentAnalysis').getOrCreate()
df_raw = spark.read.option('header', True).csv('/Users/christopherkindl/working/twitter_output_1.csv')
df_clean = df_raw.withColumn('sentiment', sentiment(df_raw['tweets']))
df_clean.show()

+---+--------------------+-------------------+-------------+---------+
|_c0|              tweets|               date|      station|sentiment|
+---+--------------------+-------------------+-------------+---------+
|  0|In #Newham today,...|2021-04-20 20:15:20|   Abbey Road|     0.62|
|  1|While there is ma...|2021-04-19 11:26:43|   Abbey Road|     0.42|
|  2|Just posted a pho...|2021-04-20 07:00:25|   Abbey Wood|      0.0|
|  3|Walking on the wi...|2021-04-21 16:20:23|Acton Central|      0.0|
|  4|Just posted a vid...|2021-04-19 17:03:53|Acton Central|      0.0|
+---+--------------------+-------------------+-------------+---------+



In [76]:
df_raw = spark.read.option('header', True).csv('/Users/christopherkindl/working/twitter_output_1.csv')

In [104]:
# convert to pandas df first to avoid folder creation which happens when using spark csv function and export to csv
header = ["tweets", "date", "station", "sentiment"]
df_clean.toPandas().to_csv('/Users/christopherkindl/desktop/twitter_output_20.csv', columns = header, index = False)

In [96]:
df_clean.write.option("header", "true").csv('/Users/christopherkindl/desktop/twitter_output_6.csv')

In [ ]:
import pyarrow.parquet as pq

In [33]:
import logging
log = logging.getLogger(__name__)

In [34]:
test = 3

In [39]:
log.info(test)

In [80]:
df = pd.read_csv("/Users/christopherkindl/Downloads/part-00000-2fa948e0-9e67-43f1-9945-e35174ac3279-c000.csv")

ParserError: Error tokenizing data. C error: Expected 4 fields in line 5, saw 5
